In [1]:
import random
import networkx as nx
from collections import deque, defaultdict
import logging
from typing import Callable

In [ ]:
#in this situation the state is the board that contains all possible actions
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [6]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state): 
            new_state = result(state, a) # this is what happens to the previous state if i apply the action a
            cost = unit_cost(a) # compute the cost
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost # append to the dict of costs the cost corresponding to the new_state
                frontier.push(new_state, p=priority_function(new_state)) # append with a specified priority function
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

[[0, 1, 5], [8, 3, 4, 7], [1, 5, 6, 7], [1, 6, 7], [0, 7], [5, 6, 7], [9, 3, 4, 6], [8, 1, 6, 0], [1], [8, 0, 2, 5], [0, 8, 2, 1], [1, 4, 5], [9, 5], [0, 9], [0, 4, 6, 7, 8], [0, 2, 7], [2, 4, 7], [8, 1, 3, 0], [8, 4], [9, 4, 7], [1, 2, 4], [1, 4, 5]]
